In [ ]:
import pandas as pd
from geopy.distance import geodesic
from datetime import timedelta
import osmnx as ox
import networkx as nx
import math

In [ ]:
pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 2.6 MB/s eta 0:00:00


In [ ]:
# Координаты бригад и предварительная загрузка данных
brigades = {
    "A": (55.8181190490723, 37.5789909362793),
    "B": (55.7531852722168, 37.4102668762207),
    "C": (55.793628692627, 37.7089691162109),
    "D": (55.6388854980469, 37.7610702514648),
    # "D_new": (55.8377838134766, 37.6557426452637)
}


In [ ]:
# Загрузите данные инцидентов
incidents = pd.read_excel('data3new.xlsx')

In [ ]:
# Обработка данных инцидентов
incidents['Широта'] = pd.to_numeric(incidents['Широта'], errors='coerce')
incidents['Долгота'] = pd.to_numeric(incidents['Долгота'], errors='coerce')
incidents = incidents.dropna(subset=['Широта', 'Долгота'])

In [ ]:
# Инициализация графа дорог для Москвы
G = ox.graph_from_place('Moscow, Russia', network_type='drive')
G = ox.project_graph(G)
G = nx.convert_node_labels_to_integers(G)

In [ ]:
def get_shortest_path_length(G, start_point, end_point, p=1):
    start_node = ox.distance.nearest_nodes(G, start_point[1], start_point[0])
    end_node = ox.distance.nearest_nodes(G, end_point[1], end_point[0])
    length = nx.shortest_path_length(G, start_node, end_node, weight='length')

    # Вычисляем расстояние между начальной и конечной точками по метрике Минковского
    start_lat, start_lon = ox.graph_to_gdfs(G, edges=False).loc[start_node][['y', 'x']].values
    end_lat, end_lon = ox.graph_to_gdfs(G, edges=False).loc[end_node][['y', 'x']].values
    minkowski_distance = (abs(start_lat - end_lat) ** p + abs(start_lon - end_lon) ** p) ** (1 / p)

    return minkowski_distance + length / 1000  # Возвращаем сумму расстояний в километрах

In [ ]:
def calculate_priority(deadline, start_time):
    # Вычисляем разницу в часах
    time_difference = (deadline - start_time).total_seconds() / 3600

    # Устанавливаем приоритет на основе разницы во времени
    if time_difference <= 4:
        return 1  # Высший приоритет для 4-часовых дедлайнов
    elif time_difference <= 6:
        return 2  # Второй по приоритету для 6-часовых дедлайнов
    else:
        return 3  # Низший приоритет для остальных

In [ ]:
# Функция для расчета времени в пути
def calculate_travel_time(distance, speed=40):
    return distance / speed  # Время в часах

In [ ]:
# Инициализация словаря загрузки бригад
brigade_workload = {brigade: 0 for brigade in brigades.keys()}

In [ ]:
# Функция для обновления загрузки бригад
def update_brigade_workload(brigade, additional_workload):
    brigade_workload[brigade] += additional_workload

In [ ]:
# def update_brigade_availability(brigade, end_time, brigade_availability):
#     brigade_availability[brigade] = max(brigade_availability[brigade], end_time)

In [ ]:
# Глобальное определение словаря доступности бригад
brigade_availability = {brigade: pd.Timestamp.min for brigade in brigades.keys()}

In [ ]:
def update_brigade_availability(brigade, end_time, brigade_availability):
    brigade_availability[brigade] = max(brigade_availability[brigade], end_time)
    return brigade_availability

In [ ]:
def some_other_function():
    # В этой функции также можно использовать brigade_availability
    global brigade_availability
    # Дальнейшая логика работы с brigade_availability

In [ ]:
# Функция назначения инцидентов бригадам
def assign_incidents_to_brigades(incidents, brigades, G, brigade_availability):
    assignments = []
    # Добавляем столбец приоритета на основе разницы между 'Крайний срок' и 'Т события'
    incidents['Priority'] = incidents.apply(lambda x: calculate_priority(x['Крайний срок'], x['Т события']), axis=1)

    # Сортируем инциденты сначала по приоритету, затем по 'Т события'
    sorted_incidents = incidents.sort_values(by=['Priority', 'Т события'])

    for _, incident in sorted_incidents.iterrows():
        incident_coord = (incident['Широта'], incident['Долгота'])
        incident_start = pd.to_datetime(incident['Т события'])
        nearest_brigade, min_distance = None, float('inf')
        best_end_time = None  # Для хранения времени окончания работы

        for brigade_name, brigade_coord in brigades.items():
            path_length = get_shortest_path_length(G, brigade_coord, incident_coord)
            travel_time_hours = calculate_travel_time(path_length)  # Время в пути в часах
            travel_time = timedelta(hours=travel_time_hours)  # Преобразуем в timedelta

            # Предполагаемое время начала работы бригады
            brigade_available_time = brigade_availability[brigade_name]
            start_work_time = max(incident_start, brigade_available_time) + travel_time

            # Добавляем 30 минут на выполнение работы
            end_work_time = start_work_time + timedelta(minutes=30)

            # Выбираем бригаду с минимальным временем окончания работы
            if best_end_time is None or end_work_time < best_end_time:
                nearest_brigade = brigade_name
                best_end_time = end_work_time

        if nearest_brigade:
            update_brigade_availability(nearest_brigade, best_end_time, brigade_availability)
            update_brigade_workload(nearest_brigade, 1)
            assignments.append((incident['№ МИ'], nearest_brigade, start_work_time, best_end_time))

    return assignments

In [ ]:
# Распределение инцидентов по дням с использованием обновленной логики
def distribute_daily_incidents(incidents, brigades, start_date, end_date, G):
    daily_assignments = {}
    current_date = start_date
    while current_date <= end_date:
        daily_incidents = incidents[incidents['Т события'].dt.date == current_date.date()]
        daily_assignments[current_date] = assign_incidents_to_brigades(daily_incidents, brigades, G, brigade_availability)  # Добавлен аргумент
        current_date += timedelta(days=1)
    return daily_assignments

In [ ]:
def assign_to_nearest_available_brigade(incidents, brigades, G):
    assignments = []
    sorted_incidents = incidents.sort_values(by='PRIORITY', ascending=False)
    for _, incident in sorted_incidents.iterrows():
        incident_coord = (incident['Широта'], incident['Долгота'])
        incident_start = incident['Т события']
        nearest_brigade, min_distance, estimated_end_time = None, float('inf'), None
        for brigade_name, brigade_coord in brigades.items():
            path_length = get_shortest_path_length(G, brigade_coord, incident_coord)
            travel_time = calculate_travel_time(path_length)
            # Расчет времени начала работы бригады с учетом времени в пути
            start_work_time = incident_start + pd.to_timedelta(travel_time, unit='h')
            # Расчет времени окончания работы (время в пути + 30 минут работы)
            end_work_time = start_work_time + pd.to_timedelta(30, unit='m')
            if path_length < min_distance:
                nearest_brigade, min_distance = brigade_name, path_length
                estimated_end_time = end_work_time
        if nearest_brigade is not None:
            update_brigade_availability(nearest_brigade, estimated_end_time)
            update_brigade_workload(nearest_brigade, 1)
            assignments.append((incident['№ МИ'], nearest_brigade, incident_start, estimated_end_time))
    return assignments

In [ ]:
# Сохранение данных
def save_assignments(daily_distributions, incidents):
    output = []
    for day, assignments in daily_distributions.items():
        for assignment in assignments:
            incident_id, brigade, start_time, end_time = assignment
            # Проверка наличия записи перед обращением
            if not incidents[incidents['№ МИ'] == incident_id].empty:
                incident_record = incidents[incidents['№ МИ'] == incident_id].iloc[0]
                output.append({
                    'Date': day.strftime('%Y-%m-%d'),
                    'Incident ID': incident_id,
                    'Assigned Brigade': brigade,
                    'Incident Type': incident_record['Техописание'],
                    'Time Assigned': incident_record['Т события'].strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(incident_record['Т события']) else 'N/A',
                    'Estimated End Time': end_time.strftime('%Y-%m-%d %H:%M:%S'),
                    'Deadline': incident_record['Крайний срок'].strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(incident_record['Крайний срок']) else 'N/A'
                })
            else:
                print(f"Warning: No incident record found for Incident ID {incident_id}. Skipping.")

    output_df = pd.DataFrame(output)
    output_df.to_csv('version_2.0.csv', index=False)

In [ ]:
# Вызов функций для распределения и сохранения результатов
start_date = incidents['Т события'].min()
end_date = incidents['Т события'].max()
daily_distributions = distribute_daily_incidents(incidents, brigades, start_date, end_date, G)
save_assignments(daily_distributions, incidents)

<ipython-input-21-cb6d091e7b08>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incidents['Priority'] = incidents.apply(lambda x: calculate_priority(x['Крайний срок'], x['Т события']), axis=1)
<ipython-input-21-cb6d091e7b08>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incidents['Priority'] = incidents.apply(lambda x: calculate_priority(x['Крайний срок'], x['Т события']), axis=1)
<ipython-input-21-cb6d091e7b08>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [ ]:
#4 часовые и 6 часовые приоретет deadline учет рабочего времени бригада A и C меняется в 9 часов утра бригады B D 8 часовые приемка час сдача смены пол часа

ValueError: not enough values to unpack (expected 4, got 2)